In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

In [ ]:
class Bitcoin(Dataset):
    
    def __init__(self, transform=None):
        df = pd.read_csv('/kaggle/input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-12-31.csv')
        df = df.dropna()
        df['Timestamp'] = df['Timestamp'] * 1e-9
        self.X, self.y = df[['Timestamp', 'Open', 'Close', 'Volume_(BTC)', 'Volume_(Currency)', 'Weighted_Price']], df[['High', 'Low']]
        self.X, self.y = torch.tensor(self.X.to_numpy(), dtype=torch.float32), torch.tensor(self.y.to_numpy(), dtype=torch.float32)
        print(df.head())
    
    def __len__(self):
        return self.y.shape[0]
    
    def __getitem__(self, index):
        sample = self.X[index], self.y[index]
        return sample

In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.hidden = 1480
        self.linear_stack = nn.Sequential(
            nn.Linear(6, self.hidden),
            nn.LeakyReLU(),
            nn.Linear(self.hidden, self.hidden),
            nn.LeakyReLU(),
            nn.Linear(self.hidden, self.hidden),
            nn.LeakyReLU(),
            nn.Linear(self.hidden, 2),
            nn.LeakyReLU()
        )
        
    def forward(self, x):
        x = self.linear_stack(x)
        return x

In [ ]:
def train(Model, dataloader):
    lr = 0.00001
    epochs = 20
    batch_size = 256
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(Model.parameters(), lr=lr)
    size = len(dataloader.dataset)
    
    for epoch in range(epochs):
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            pred = Model(X)
            loss = loss_fn(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if batch % 3500 == 0:
                loss, current = loss.item(), batch * len(X)
                print(f"epoch: {epoch + 1} [{current}/{size}] loss: {loss}")

            
def test(Model, dataloader):
    batch_size = 256
    loss_fn = nn.MSELoss()
    size = len(dataloader.dataset)
    test_loss = 0 
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = Model(X)
            test_loss += loss_fn(pred, y).item()
            #print(pred, y)
            #print(y-pred)
            #break
        test_loss /= size
        print(f"Avg loss: {test_loss}")
            

In [ ]:
dataset = Bitcoin()

train_loader = DataLoader(dataset=dataset, batch_size=256, shuffle=True)

test_loader = DataLoader(dataset=dataset, batch_size=256, shuffle=False)

In [ ]:
Model = Net().to(device)
Model

In [ ]:
train(Model, train_loader)

In [ ]:
test(Model, test_loader)

In [ ]:
torch.save(Model.state_dict(), 'Model.pth')
print("Model saved")

In [ ]:
Model = Net().to(device)
Model.load_state_dict(torch.load('Model.pth'))
Model.eval()